In [17]:
import pandas as pd
import os
from crewai import Crew, Agent, LLM, Task, Process
from crewai_tools import BaseTool, PDFSearchTool
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.agents import Tool
from langchain.embeddings.openai import OpenAIEmbeddings
import openai
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from crewai_tools import BaseTool
from langchain_core.callbacks import BaseCallbackHandler
from typing import Any, Dict

import warnings
warnings.filterwarnings('ignore')

In [3]:
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://genai-openai-ai-mazinghacktivists.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "66243e4ca95e4b46ae6b8a7cc29c7517"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-05-15"

In [4]:
# Load PDFs
pdf_dir = r"C:\Users\SGouravaram\Downloads\datasets"
loader = PyPDFDirectoryLoader(pdf_dir)
docs = loader.load()

In [5]:
# split the doc into smaller chunks i.e. chunk_size=250
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=25)
chunks = text_splitter.split_documents(docs)

# get OpenAI Embedding model
embeddings = AzureOpenAIEmbeddings(model="text-embedding-3-large")

# embed the chunks as vectors and load them into the database
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db_pdfs")
db_chroma.persist()

In [6]:
# Define retriever from vector store
retriever = db_chroma.as_retriever()

In [7]:
class GetRelevantDoc(BaseTool):
    name: str = "Doc_search"
    description: str = "useful to get relevant data from the pdf data"

    def _run(self, query: str) -> str:
        return retriever.get_relevant_documents(query)


tools=[GetRelevantDoc()]

In [8]:
llm_4o = LLM(
    model="azure/gpt-4o",
    base_url="https://genai-openai-ai-mazinghacktivists.openai.azure.com/",
    api_key="66243e4ca95e4b46ae6b8a7cc29c7517"
)

In [2]:
class CustomHandler(BaseCallbackHandler):
    """A custom handler for logging interactions within the process chain."""
    
    def __init__(self, agent_name: str) -> None:
        super().__init__()
        self.agent_name = agent_name

    def on_chain_start(self, serialized: Dict[str, Any], outputs: Dict[str, Any], **kwargs: Any) -> None:
        """Log the start of a chain with user input."""
        st.session_state.messages.append({"role": "assistant", "content": outputs['input']})
        st.chat_message("assistant").write(outputs['input'])
        
    def on_agent_action(self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any) -> None:
        """""Log the action taken by an agent during a chain run."""
        st.session_state.messages.append({"role": "assistant", "content": inputs['input']})
        st.chat_message("assistant").write(inputs['input'])
        
    def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> None:
        """Log the end of a chain with the output generated by an agent."""
        st.session_state.messages.append({"role": self.agent_name, "content": outputs['output']})
        st.chat_message(self.agent_name).write(outputs['output'])

In [9]:
business_agent = Agent(
    role="Senior Business Analyst",
    goal="Identify key risks and RWA optimization opportunities.",
    backstory="""You are a senior business analyst in barclays credit risk department. You are responsible for analyzing RWA related data for optimization of RWA.
    You are currently working on  a project to review the business data, check for any discrepancies, and suggest adjustments that could reduce RWA. You should
    focus on asset classifications, exposure amounts, and credit ratings.
    """,
    allow_delegation=False,
    llm=llm_4o,
    callbacks=[CustomHandler("Senior Business Analyst")],
    tools=tools
)

In [10]:
data_standardization_agent = Agent(
    role="Senior Data Standardization Advisor",
    goal="Ensure all reference data is standardized",
    backstory="""You are a senior  Data Standardization Advisor in barclays credit risk department. You are responsible for ensuring that all reference data
    is consistent across different formats. You are currently working on a project to convert all currency, date, asset type etc to a standard model
    to ensure correct risk-weighting classifications.
    """,
    allow_delegation=False,
    llm=llm_4o,
    callbacks=[CustomHandler("Senior Data Standardization Advisor")],
    tools=tools
)

In [11]:
anomaly_detection_agent = Agent(
    role="Senior Anomaly Detection Analyst",
    goal="Flag inconsistencies and outliers that could impact RWA.",
    backstory="""You are a senior Anomaly Detection Analyst in barclays credit risk department. You are responsible to identify any irregularies or anomalies
    in the data. You are currently working on a project to look for outlier values in exposure, credit ratings, and other key RWA metrics that could indicate data 
    issue and adversly impact RWA.
    """,
    allow_delegation=False,
    llm=llm_4o,
    callbacks=[CustomHandler("Senior Anomaly Detection Analyst")],
    tools=tools
)

In [12]:
customer_segmentation_agent = Agent(
    role="Senior Customer Segmentation Analyst",
    goal="Segment customer on the basis of risk profile, credit ratings etc",
    backstory="""You are a senior Customer Segmentation Analyst in barclays credit risk department. You are responsible to provide RWA optimization using 
    segmentation and capital optimization strategies based on customer profiles. You are currently working on a project to segment customers on the basis of
    credit ratings, loan types, industry and loan term and provide strategy to optimize RWA.
    """,
    allow_delegation=False,
    llm=llm_4o,
    callbacks=[CustomHandler("Senior Customer Segmentation Analyst")],
    tools=tools
)

In [13]:
rwa_optimization_agent = Agent(
    role="RWA Optimization Advisor",
    goal="Offers targeted recommendations to reduce RWA and suggest optimized capital allocation.",
    backstory="""You are a senior RWA Optimization Advisor at Barclays Bank PLC, skilled in providing RWA Optimization 
    strategies. Your job  is to provide strategies for minimizing RWA, recommend actions such as asset reclassification, securitization ,
    or reducing exposure high-risk assets. Besides, your job is to suggest optimized capital allocation to minimize RWA while managing risk exposure.
    """,
    allow_delegation=False,
    llm=llm_4o,
    tools=tools
)

In [14]:
rwa_optimization_task = Task(
    description="Summarize all the RWA optimization suggestion",
    expected_output="A bullet list summary of top 10 most important RWA and capital allocation Optimization and all the points are explained properly.",
    agent=rwa_optimization_agent
)

In [15]:
prompt= "Barclays wants to trade in India real estate sector with exposure of INR 1 billion. Real Estate customer has PD as 6% and LGD as 20%. Please suggest RWA optimization strategy adhering to Basel 3 regulations."

business_analysis_task = Task(
    description=f"""Analyze the user input '{prompt}' and identify key risks and RWA optimization opportunities.""",
    expected_output="A bullet list summary of top 3 most important RWA Optimization opportunities.",
    agent=business_agent
)

data_standardization_task = Task(
    description=f"""Analyze the user input '{prompt}', and identify reference data standardization opportunity 
    so that correct risk weighting classification can be done.""",
    expected_output="A bullet list summary of top 3 most important reference data standardization opportunity.",
    agent=data_standardization_agent
)

anomaly_detection_task = Task(
    description=f"""Analyze the user input '{prompt}', and flag inconsistencies and outliers that could adversly impact RWA.""",
    expected_output="A bullet list summary of top 3 most important anomalies related with key RWA metrics.",
    agent=anomaly_detection_agent
)

customer_segmentation_task = Task(
    description=f"""Analyze the user input '{prompt}', and provide suggestions for RWA and capital optimization opportunities using customer segmentation.""",
    expected_output="A bullet list summary of top 3 most important RWA and capital optimization opportunities using customer segmentation. ",
    agent=customer_segmentation_agent
)

In [18]:
# Create Crew
rwaoptima_crew1 = Crew(
    agents=[business_agent,data_standardization_agent,anomaly_detection_agent,customer_segmentation_agent],
    tasks=[business_analysis_task,data_standardization_task,anomaly_detection_task,customer_segmentation_task],
    process=Process.hierarchical,
    manager_llm=llm_4o,
    verbose=True  # This will print logs to the console as the crew works
)

# Job Context
#job_crew_works = {
#    'user_input': 'Barclays wants to trade in India real estate sector with exposure of INR 1 billion. Real Estate customer has PD as 6% and LGD as 20%. Please suggest RWA optimization strategy adhering to Basel 3 regulations.'
#}

prompt = "Barclays wants to trade in India real estate sector with exposure of INR 1 billion. Real Estate customer has PD as 6% and LGD as 20%. Please suggest RWA optimization strategy adhering to Basel 3 regulations."

# Kickoff the Crew's Work
result = rwaoptima_crew1.kickoff()
print(result)

# Agent: Crew Manager
## Task: Analyze the user input 'Barclays wants to trade in India real estate sector with exposure of INR 1 billion. Real Estate customer has PD as 6% and LGD as 20%. Please suggest RWA optimization strategy adhering to Basel 3 regulations.' and identify key risks and RWA optimization opportunities.
# Agent: Senior Business Analyst
## Task: Analyze the user input 'Barclays wants to trade in India real estate sector with exposure of INR 1 billion. Real Estate customer has PD as 6% and LGD as 20%. Please suggest RWA optimization strategy adhering to Basel 3 regulations.' and identify key risks and RWA optimization opportunities.


# Agent: Senior Business Analyst
## Thought: To provide a comprehensive response to the user input and identify key risks and RWA optimization opportunities, I'll first need to gather detailed information on Basel 3 regulations, focusing on the real estate sector. This will enable me to suggest appropriate strategies for optimizing RWA and